In [2]:
##importing Basic libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import numpy as np
#import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
import bokeh as bk
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error,r2_score
import plotly
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
import pybaseball as pyb
from sklearn.linear_model import ElasticNet
from pybaseball import statcast
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter
from pybaseball import statcast_pitcher_exitvelo_barrels
from pybaseball import statcast_batter_exitvelo_barrels
from pybaseball import statcast_batter_expected_stats
from pybaseball import statcast_pitcher_expected_stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pwlf as pwlf
#import matplotlib.backends
import matplotlib as mpl
import joblib
import pickle as pkl

2025-05-09 10:53:55.203093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from datetime import datetime, timedelta
import warnings

all_time_path = "/Users/johndavis/Desktop/Desktop_2/Statcast_ALL_TIME.csv"
current_year_path = "/Users/johndavis/Desktop/Statcast_2024.csv"

df_2022 = pd.read_csv(all_time_path, low_memory=False)
df_2023 = pd.read_csv(current_year_path, low_memory=False)

for df in [df_2022, df_2023]:
    df['pfx_x'] = df['pfx_x'] * -12  # Convert to inches and flip perspective
    df['pfx_z'] = df['pfx_z'] * 12
    df['Year'] = df['game_date'].astype(str).str[:4]

rename_cols = {
    'game_date': 'Date', 'player_name': 'Pitcher', 'pitcher': 'Pitcher_ID',
    'p_throws': 'Hand', 'pitch_type': 'TaggedPitchType', 'release_speed': 'RelSpeed',
    'release_pos_x': 'RelSide', 'release_pos_z': 'RelHeight', 'pfx_x': 'HorzBreak',
    'pfx_z': 'InducedVertBreak', 'release_extension': 'Extension',
    'plate_x': 'PlateLocSide', 'plate_z': 'PlateLocHeight', 'release_spin_rate': 'SpinRate'
}

df_2022.rename(columns=rename_cols, inplace=True)
df_2023.rename(columns=rename_cols, inplace=True)
df_2022['events'].fillna(df_2022['description'], inplace=True)
df_2023['events'].fillna(df_2023['description'], inplace=True)

df_2022['PitcherTeam'] = np.where(df_2022['inning_topbot'] == 'Bot', df_2022['away_team'], df_2022['home_team'])
df_2023['PitcherTeam'] = np.where(df_2023['inning_topbot'] == 'Bot', df_2023['away_team'], df_2023['home_team'])

df_2022 = df_2022[['Year', 'Pitcher', 'Hand', 'TaggedPitchType', 'events', 'description', 
                   'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'RelSide', 
                   'RelHeight', 'Extension', 'PlateLocSide', 'PlateLocHeight', 'batter', 
                   'launch_speed', 'launch_angle', 'hit_distance_sc', 'delta_run_exp']]

df_2023 = df_2023[['Year', 'Pitcher', 'Pitcher_ID', 'PitcherTeam', 'Hand', 'TaggedPitchType', 
                   'events', 'description', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 
                   'HorzBreak', 'RelSide', 'RelHeight', 'Extension', 'PlateLocSide', 
                   'PlateLocHeight', 'batter', 'launch_speed', 'launch_angle', 
                   'hit_distance_sc', 'delta_run_exp', 'bat_speed']]

df_2022['RunValue'] = df_2022['delta_run_exp']
df_2023['RunValue'] = df_2023['delta_run_exp']
df_2022['RunValue'].fillna(0, inplace=True)
df_2023['RunValue'].fillna(0, inplace=True)


df_2022_hit_into_play = df_2022[df_2022['description'] == 'hit_into_play']
df_2023_hit_into_play = df_2023[df_2023['description'] == 'hit_into_play']
df_not_hit_into_play = df_2022[df_2022['description'] != 'hit_into_play']
df_not_hit_into_play_2023 = df_2023[df_2023['description'] != 'hit_into_play']

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1124706807.py:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1124706807.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
X = df_2022_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']]
y = df_2022_hit_into_play['RunValue']
X1 = df_2023_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']]
y1 = df_2023_hit_into_play['RunValue']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X1_train, y1_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}

xgb_model = XGBRegressor(**params)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
xgb_model_1 = XGBRegressor(**params)
xgb_model_1.fit(X1_train, y1_train, eval_set=[(X_val_1, y_val_1)], early_stopping_rounds=10, verbose=False)
from sklearn.utils import resample
from sklearn.metrics import r2_score

n_estimators = 10  
predictions = []


for i in range(n_estimators):
    X_train_sample, y_train_sample = resample(X_train, y_train)
    model = XGBRegressor(**params)
    model.fit(X_train_sample, y_train_sample, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
    preds = model.predict(X_val)
    predictions.append(preds)
    
ensemble_preds = np.mean(predictions, axis=0)
ensemble_r2 = r2_score(y_val, ensemble_preds)
dtrain = xgb.DMatrix(X_train, label=y_train)
cv_results = xgb.cv(
    dtrain=dtrain, 
    params=params, 
    nfold=5, 
    num_boost_round=200, 
    early_stopping_rounds=10, 
    metrics="rmse",
    as_pandas=True, 
    seed=42
)
variance_y = y_train.var()
best_r_squared = 1 - (cv_results['test-rmse-mean'].min() ** 2) / variance_y
print('xRun Value EV/LA Best  R^2:', '{:.20f}'.format(best_r_squared))

/Users/johndavis/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning:

`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.



[11:05:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[11:05:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[11:05:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[11:05:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[11:05:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

xRun Value EV/LA Best  R^2: 0.37585376980684326309


In [22]:
df_2022_hit_into_play['RunValue'] = xgb_model.predict(df_2022_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']])
df_2023_hit_into_play['RunValue'] = xgb_model_1.predict(df_2023_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']])

#combine df_2022_hit_into_play and df_not_hit_into_play 
df_2022_1 = pd.concat([df_2022_hit_into_play,df_not_hit_into_play ],ignore_index=True)
df_2023_1 = pd.concat([df_2023_hit_into_play,df_not_hit_into_play_2023 ],ignore_index=True)
df_2022 = df_2022_1
df_2023 = df_2023_1
df_2023['TaggedPitchType'].unique()
#remove any Nan RunValue
df_2022 = df_2022[df_2022['RunValue'].notna()]
df_2023 = df_2023[df_2023['RunValue'].notna()]

#change pitch type to match trackman data
#FF, SI, FS,FA  = Fastball
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FF','FA'],'Fastball')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FF','FA'],'Fastball')

df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SI'], 'Sinker')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SI'], 'Sinker')

#SL,ST,FC  = Slider
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SL'],'Slider')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SL'],'Slider')

df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['ST'],'Sweeper')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['ST'],'Sweeper')

#CH, EP, SC, FO = ChangeUp
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['CH','EP','SC','FO','KN'],'ChangeUp')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['CH','EP','SC','FO','KN'],'ChangeUp')
#SV, CU, KC = Curveball
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SV','CU','KC'],'Curveball')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SV','CU','KC'],'Curveball')
#'FC' = Cutter
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FC'],'Cutter')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FC'],'Cutter')
#SPL = FS
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FS'],'Splitter')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FS'],'Splitter')

fastball_pool = ['Fastball', 'FourSeamFastball', 'Sinker', 'TwoSeamFastBall', 'OneSeamFastBall']

def get_primary_fastball_velo(df):
    fb_df = df[df['TaggedPitchType'].isin(fastball_pool)]
    pitch_counts = fb_df.groupby(['Pitcher', 'TaggedPitchType']).size().reset_index(name='Count')
    primary_pitch = pitch_counts.loc[pitch_counts.groupby('Pitcher')['Count'].idxmax()].reset_index(drop=True)
    avg_velos = fb_df.groupby(['Pitcher', 'TaggedPitchType'])['RelSpeed'].mean().reset_index()
    primary_velos = pd.merge(primary_pitch, avg_velos, on=['Pitcher', 'TaggedPitchType'], how='left')
    primary_velos = primary_velos.rename(columns={'RelSpeed': 'PrimaryFB_Velo'})
    
    return primary_velos[['Pitcher', 'PrimaryFB_Velo']]

primary_fb_2022 = get_primary_fastball_velo(df_2022)
primary_fb_2023 = get_primary_fastball_velo(df_2023)

df_2022 = df_2022.merge(primary_fb_2022, on='Pitcher', how='left')
df_2023 = df_2023.merge(primary_fb_2023, on='Pitcher', how='left')
df_2022['VeloDiff_FF'] = df_2022['RelSpeed'] - df_2022['PrimaryFB_Velo']
df_2023['VeloDiff_FF'] = df_2023['RelSpeed'] - df_2023['PrimaryFB_Velo']


df_2022['ABS_Horizontal'] = abs(df_2022['HorzBreak'])
df_2023['ABS_Horizontal'] = abs(df_2023['HorzBreak'])
df_2022['ABS_RelSide'] = abs(df_2022['RelSide'])
df_2023['ABS_RelSide'] = abs(df_2023['RelSide'])
df_2022['differential_break'] = abs(df_2022['InducedVertBreak'] - df_2022['ABS_Horizontal'])
df_2023['differential_break'] = abs(df_2023['InducedVertBreak'] - df_2023['ABS_Horizontal'])

df_2022['ABS_Horizontal'] = abs(df_2022['HorzBreak'])
df_2023['ABS_Horizontal'] = abs(df_2023['HorzBreak'])
df_2022['ABS_RelSide'] = abs(df_2022['RelSide'])
df_2023['ABS_RelSide'] = abs(df_2023['RelSide'])
df_2022['differential_break'] = abs(df_2022['InducedVertBreak'] - df_2022['ABS_Horizontal'])
df_2023['differential_break'] = abs(df_2023['InducedVertBreak'] - df_2023['ABS_Horizontal'])


#fastballs = 'Fastball', 'Sinker', 'TwoSeamFastBall', 'FourSeamFastBall', 'OneSeamFastBall'
dfb2 = df_2022[df_2022.TaggedPitchType.isin(['Fastball', 'FourSeamFastball', 'OneSeamFastBall'])]
dfb3 = df_2023[df_2023.TaggedPitchType.isin(['Fastball', 'FourSeamFastball', 'OneSeamFastBall'])]
#'Sinker', 'TwoSeamFastBall',
dsi2 = df_2022[df_2022.TaggedPitchType.isin(['Sinker', 'TwoSeamFastBall'])]
dsi3 = df_2023[df_2023.TaggedPitchType.isin(['Sinker', 'TwoSeamFastBall'])]
#sliders = 'Slider', 'Cutter'
dsl2 = df_2022[df_2022.TaggedPitchType.isin(['Slider'])]
dsl3 = df_2023[df_2023.TaggedPitchType.isin(['Slider'])]
#Sweeper
dst2 = df_2022[df_2022.TaggedPitchType.isin(['Sweeper'])]
dst3 = df_2023[df_2023.TaggedPitchType.isin(['Sweeper'])]
#curveballs = 'Curveball', 'KnuckleCurve'
dcb2 = df_2022[df_2022.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
dcb3 = df_2023[df_2023.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
#changeups = 'Changeup', 'Splitter', 'Forkball', 'Screwball'
dch2 = df_2022[df_2022.TaggedPitchType.isin(['ChangeUp'])]
dch3 = df_2023[df_2023.TaggedPitchType.isin(['ChangeUp'])]
#cutters = 'Cutter'
dct2 = df_2022[df_2022.TaggedPitchType.isin(['Cutter'])]
dct3 = df_2023[df_2023.TaggedPitchType.isin(['Cutter'])]
#splitter
dsp2 = df_2022[df_2022.TaggedPitchType.isin(['Splitter'])]
dsp3 = df_2023[df_2023.TaggedPitchType.isin(['Splitter'])]
#delete all rows with nan values
dfb2 = dfb2.dropna()
dsi2 = dsi2.dropna()
dsl2 = dsl2.dropna()
dst2 = dst2.dropna()
dcb2 = dcb2.dropna()
dch2 = dch2.dropna()
dct2 = dct2.dropna()
dsp2 = dsp2.dropna()

In [35]:
#Bat Speed Model
df_2024 = pd.read_csv("/Users/johndavis/Desktop/Statcast_2024.csv",low_memory=False)
df_2024['Year'] = df_2024['game_date'].astype(str).str[:4]
df_2024['pfx_z'] = df_2024['pfx_z']*12
df_2024['pfx_x'] = df_2024['pfx_x']*12
df_2024['pfx_x'] = df_2024['pfx_x']*-1
df_2024['events'].fillna(df_2024['description'], inplace=True)
df_2024.rename(columns={'game_date': 'Date', 'player_name':'Pitcher','pitcher':'Pitcher_ID','p_throws':'Hand','pitch_type':'TaggedPitchType','release_speed':'RelSpeed','release_pos_x':'RelSide','release_pos_z':'RelHeight','pfx_x':'HorzBreak','pfx_z':'InducedVertBreak','release_extension':'Extension','plate_x': 'PlateLocSide','plate_z': 'PlateLocHeight','release_spin_rate':'SpinRate'},inplace=True)
# PITCH NAMES
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['FF','FA'],'Fastball')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['SI'], 'Sinker')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['SL'],'Slider')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['ST'],'Sweeper')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['CH','EP','SC','FO','KN'],'ChangeUp')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['SV','CU','KC'],'Curveball')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['FC'],'Cutter')
df_2024['TaggedPitchType'] = df_2024['TaggedPitchType'].replace(['FS'],'Splitter')
df_2024['ABS_Horizontal'] = abs(df_2024['HorzBreak'])
df_2024['ABS_RelSide'] = abs(df_2024['RelSide'])
df_2024['differential_break'] = abs(df_2024['InducedVertBreak'] - df_2024['ABS_Horizontal'])

df_2024 = df_2024[['Year','Pitcher','Pitcher_ID','Hand','TaggedPitchType','events','description','RelSpeed','SpinRate','InducedVertBreak','HorzBreak','RelSide','RelHeight','ABS_Horizontal','ABS_RelSide','differential_break','Extension','PlateLocSide','PlateLocHeight','batter','launch_speed','launch_angle','hit_distance_sc','delta_run_exp','bat_speed']]

dfb2024 = df_2024[df_2024.TaggedPitchType.isin(['Fastball', 'FourSeamFastball', 'OneSeamFastBall'])]
dsi2024 = df_2024[df_2024.TaggedPitchType.isin(['Sinker', 'TwoSeamFastBall'])]
dsl2024 = df_2024[df_2024.TaggedPitchType.isin(['Slider'])]
dst2024 = df_2024[df_2024.TaggedPitchType.isin(['Sweeper'])]
dcb2024 = df_2024[df_2024.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
dch2024 = df_2024[df_2024.TaggedPitchType.isin(['ChangeUp'])]
dct2024 = df_2024[df_2024.TaggedPitchType.isin(['Cutter'])]
dsp2024 = df_2024[df_2024.TaggedPitchType.isin(['Splitter'])]
#remove if bat_speed is nan specifically that column
dfb2024 = dfb2024.dropna(subset=['bat_speed'])
dsi2024 = dsi2024.dropna(subset=['bat_speed'])
dsl2024 = dsl2024.dropna(subset=['bat_speed'])
dst2024 = dst2024.dropna(subset=['bat_speed'])
dcb2024 = dcb2024.dropna(subset=['bat_speed'])
dch2024 = dch2024.dropna(subset=['bat_speed'])
dct2024 = dct2024.dropna(subset=['bat_speed'])
dsp2024 = dsp2024.dropna(subset=['bat_speed'])

X_Batspeed_fb = dfb2024[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_fb = dfb2024['bat_speed']
X_Batspeed_SI = dsi2024[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_SI = dsi2024['bat_speed']
X_Batspeed_SL = dsl2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_SL = dsl2024['bat_speed']
X_Batspeed_SW = dst2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_SW = dst2024['bat_speed']
X_Batspeed_CB = dcb2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_CB = dcb2024['bat_speed']
X_Batspeed_CH = dch2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_CH = dch2024['bat_speed']
X_Batspeed_CT = dct2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_CT = dct2024['bat_speed']
X_Batspeed_SP = dsp2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y_Batspeed_SP = dsp2024['bat_speed']
#test train split
X_Batspeed_fb_train, X_Batspeed_fb_test, y_Batspeed_fb_train, y_Batspeed_fb_test = train_test_split(X_Batspeed_fb, y_Batspeed_fb, test_size=0.2, random_state=42)
X_Batspeed_SI_train, X_Batspeed_SI_test, y_Batspeed_SI_train, y_Batspeed_SI_test = train_test_split(X_Batspeed_SI, y_Batspeed_SI, test_size=0.2, random_state=42)
X_Batspeed_SL_train, X_Batspeed_SL_test, y_Batspeed_SL_train, y_Batspeed_SL_test = train_test_split(X_Batspeed_SL, y_Batspeed_SL, test_size=0.2, random_state=42)
X_Batspeed_SW_train, X_Batspeed_SW_test, y_Batspeed_SW_train, y_Batspeed_SW_test = train_test_split(X_Batspeed_SW, y_Batspeed_SW, test_size=0.2, random_state=42)
X_Batspeed_CB_train, X_Batspeed_CB_test, y_Batspeed_CB_train, y_Batspeed_CB_test = train_test_split(X_Batspeed_CB, y_Batspeed_CB, test_size=0.2, random_state=42)
X_Batspeed_CH_train, X_Batspeed_CH_test, y_Batspeed_CH_train, y_Batspeed_CH_test = train_test_split(X_Batspeed_CH, y_Batspeed_CH, test_size=0.2, random_state=42)
X_Batspeed_CT_train, X_Batspeed_CT_test, y_Batspeed_CT_train, y_Batspeed_CT_test = train_test_split(X_Batspeed_CT, y_Batspeed_CT, test_size=0.2, random_state=42)
X_Batspeed_SP_train, X_Batspeed_SP_test, y_Batspeed_SP_train, y_Batspeed_SP_test = train_test_split(X_Batspeed_SP, y_Batspeed_SP, test_size=0.2, random_state=42)
#train model
model_fb = XGBRegressor(**params)
model_fb.fit(X_Batspeed_fb_train, y_Batspeed_fb_train, eval_set=[(X_Batspeed_fb_test, y_Batspeed_fb_test)], verbose=False)
model_SI = XGBRegressor(**params)
model_SI.fit(X_Batspeed_SI_train, y_Batspeed_SI_train, eval_set=[(X_Batspeed_SI_test, y_Batspeed_SI_test)], verbose=False)
model_SL = XGBRegressor(**params)
model_SL.fit(X_Batspeed_SL_train, y_Batspeed_SL_train, eval_set=[(X_Batspeed_SL_test, y_Batspeed_SL_test)], verbose=False)
model_SW = XGBRegressor(**params)
model_SW.fit(X_Batspeed_SW_train, y_Batspeed_SW_train, eval_set=[(X_Batspeed_SW_test, y_Batspeed_SW_test)], verbose=False)
model_CB = XGBRegressor(**params)
model_CB.fit(X_Batspeed_CB_train, y_Batspeed_CB_train, eval_set=[(X_Batspeed_CB_test, y_Batspeed_CB_test)], verbose=False)
model_CH = XGBRegressor(**params)
model_CH.fit(X_Batspeed_CH_train, y_Batspeed_CH_train, eval_set=[(X_Batspeed_CH_test, y_Batspeed_CH_test)], verbose=False)
model_CT = XGBRegressor(**params)
model_CT.fit(X_Batspeed_CT_train, y_Batspeed_CT_train, eval_set=[(X_Batspeed_CT_test, y_Batspeed_CT_test)], verbose=False)
model_SP = XGBRegressor(**params)
model_SP.fit(X_Batspeed_SP_train, y_Batspeed_SP_train, eval_set=[(X_Batspeed_SP_test, y_Batspeed_SP_test)], verbose=False)
#predict over each pitch type
dfb2024['xbat_speed'] = model_fb.predict(dfb2024[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsi2024['xbat_speed'] = model_SI.predict(dsi2024[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsl2024['xbat_speed'] = model_SL.predict(dsl2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dst2024['xbat_speed'] = model_SW.predict(dst2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dcb2024['xbat_speed'] = model_CB.predict(dcb2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dch2024['xbat_speed'] = model_CH.predict(dch2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dct2024['xbat_speed'] = model_CT.predict(dct2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dsp2024['xbat_speed'] = model_SP.predict(dsp2024[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])

#combine all dataframes
df_2024 = pd.concat([dfb2024, dsi2024, dsl2024, dst2024, dcb2024, dch2024, dct2024, dsp2024], ignore_index=True)
#group xbat_speed by pitcher and sort by descending
df_2024_grouped = df_2024.groupby(['Pitcher']).agg({'xbat_speed':'mean','bat_speed': 'mean'}).reset_index()
#sort by descending
df_2024_grouped = df_2024_grouped.sort_values(by=['Pitcher','xbat_speed'], ascending=[True,False])

dfb2['xBatSpeed'] = model_fb.predict(dfb2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsi2['xBatSpeed'] = model_SI.predict(dsi2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsl2['xBatSpeed'] = model_SL.predict(dsl2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dst2['xBatSpeed'] = model_SW.predict(dst2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dcb2['xBatSpeed'] = model_CB.predict(dcb2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dch2['xBatSpeed'] = model_CH.predict(dch2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dct2['xBatSpeed'] = model_CT.predict(dct2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dsp2['xBatSpeed'] = model_SP.predict(dsp2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dfb3['xBatSpeed'] = model_fb.predict(dfb3[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsi3['xBatSpeed'] = model_SI.predict(dsi3[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsl3['xBatSpeed'] = model_SL.predict(dsl3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dst3['xBatSpeed'] = model_SW.predict(dst3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dcb3['xBatSpeed'] = model_CB.predict(dcb3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dch3['xBatSpeed'] = model_CH.predict(dch3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dct3['xBatSpeed'] = model_CT.predict(dct3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])
dsp3['xBatSpeed'] = model_SP.predict(dsp3[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']])


In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from pygam import LinearGAM, s
from sklearn.metrics import r2_score
import optuna
import joblib
optuna.logging.set_verbosity(optuna.logging.WARNING)
pitch_datasets = [
    ("dfb", dfb2, dfb3, ['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed']),
    ("dsi", dsi2, dsi3, ['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed']),
    ("dsl", dsl2, dsl3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF']),
    ("dst", dst2, dst3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF']),
    ("dcb", dcb2, dcb3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF']),
    ("dch", dch2, dch3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF']),
    ("dct", dct2, dct3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF']),
    ("dsp", dsp2, dsp3, ['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension','xBatSpeed','VeloDiff_FF'])
]

models = []

for label, df2, df3, features in pitch_datasets:
    print(f"{label}2 missing values: {df2[features].isna().sum().sum()}")
    print(f"{label}3 missing values: {df3[features].isna().sum().sum()}")

    X = df2[features].dropna()
    y = df2['RunValue'].loc[X.index].dropna()

    def objective(trial):
        model = XGBRegressor(
            max_depth=trial.suggest_int('max_depth', 4, 10),
            n_estimators=trial.suggest_int('n_estimators', 100, 300),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
            alpha=trial.suggest_float('alpha', 0.0, 0.3),
            reg_lambda=trial.suggest_float('reg_lambda', 0.5, 3.0),
            objective='reg:squarederror',
            early_stopping_rounds=10,
            n_jobs=-1
        )
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds = model.predict(X_val)
        return r2_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, n_jobs=4)  
    best_params = study.best_params
    print(f"Best parameters for {label}: {best_params}")

    model = XGBRegressor(**best_params, objective='reg:squarederror', n_jobs=-1)
    model.fit(X, y)
    df2['xRV_xgb'] = pd.Series(model.predict(X), index=X.index)
    df3_features = df3[features].dropna()
    df3['xRV_xgb'] = pd.Series(model.predict(df3_features), index=df3_features.index)

    residuals = y - model.predict(X)
    spline_terms = s(0)
    for i in range(1, X.shape[1]):
        spline_terms += s(i)
    gam = LinearGAM(spline_terms, lam=0.3)  
    gam.fit(X, residuals)
    df2['xRV_gam'] = pd.Series(gam.predict(X), index=X.index)
    df3['xRV_gam'] = pd.Series(gam.predict(df3_features), index=df3_features.index)

    df2['xRV_ensemble'] = df2['xRV_xgb'] + df2['xRV_gam']
    df3['xRV_ensemble'] = df3['xRV_xgb'] + df3['xRV_gam']

    for df in [df2, df3]:
        max_val = df['xRV_ensemble'].max()
        df['xRV_Scaled'] = df['xRV_ensemble'] - max_val
        df['xRV/100_stuff_scaled_abs'] = df['xRV_Scaled'].abs()
        df['Stuff_plus'] = df['xRV/100_stuff_scaled_abs'] / df['xRV/100_stuff_scaled_abs'].mean() * 100

    models.append(model)

dfb2 missing values: 0
dfb3 missing values: 1108
Best parameters for dfb: {'max_depth': 5, 'n_estimators': 226, 'learning_rate': 0.18091442168277586, 'subsample': 0.8936693062888241, 'colsample_bytree': 0.8648166461844193, 'alpha': 0.1894747955504935, 'reg_lambda': 1.605555338483215}


/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

dsi2 missing values: 0
dsi3 missing values: 941
Best parameters for dsi: {'max_depth': 5, 'n_estimators': 256, 'learning_rate': 0.2357057033100282, 'subsample': 0.8687644947664707, 'colsample_bytree': 0.630291094119466, 'alpha': 0.06874155981391335, 'reg_lambda': 1.3333964714650368}


/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

dsl2 missing values: 0
dsl3 missing values: 569
Best parameters for dsl: {'max_depth': 5, 'n_estimators': 282, 'learning_rate': 0.11746415087231682, 'subsample': 0.8074141898318111, 'colsample_bytree': 0.7474446964440556, 'alpha': 0.2885636225240698, 'reg_lambda': 2.3076980924375055}


...

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

dct2 missing values: 0
dct3 missing values: 416
Best parameters for dct: {'max_depth': 4, 'n_estimators': 161, 'learning_rate': 0.2123047908174315, 'subsample': 0.9724112598625044, 'colsample_bytree': 0.7298354893510441, 'alpha': 0.26623144136884064, 'reg_lambda': 1.7442461736135122}


/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

dsp2 missing values: 0
dsp3 missing values: 94
Best parameters for dsp: {'max_depth': 4, 'n_estimators': 246, 'learning_rate': 0.100266583725195, 'subsample': 0.6340272688148503, 'colsample_bytree': 0.6546114643978138, 'alpha': 0.23243069027910407, 'reg_lambda': 2.748889249307441}


/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/1371713061.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [48]:
current_std_fb = dfb3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_fb
mean = dfb3['Stuff_plus'].mean()
dfb3['Final_Adjusted_Stuff_Plus'] = ((dfb3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dfb3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dfb3['Final_Adjusted_Stuff_Plus'].mean()

current_std_si = dsi3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_si
mean = dsi3['Stuff_plus'].mean()
dsi3['Final_Adjusted_Stuff_Plus'] = ((dsi3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dsi3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dsi3['Final_Adjusted_Stuff_Plus'].mean()

current_std_sl = dsl3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_sl
mean = dsl3['Stuff_plus'].mean()
dsl3['Final_Adjusted_Stuff_Plus'] = ((dsl3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dsl3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dsl3['Final_Adjusted_Stuff_Plus'].mean()

current_std_st = dst3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_st
mean = dst3['Stuff_plus'].mean()
dst3['Final_Adjusted_Stuff_Plus'] = ((dst3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dst3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dst3['Final_Adjusted_Stuff_Plus'].mean()

current_std_cb = dcb3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_cb
mean = dcb3['Stuff_plus'].mean()
dcb3['Final_Adjusted_Stuff_Plus'] = ((dcb3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dcb3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dcb3['Final_Adjusted_Stuff_Plus'].mean()

current_std_ch = dch3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_ch
mean = dch3['Stuff_plus'].mean()
dch3['Final_Adjusted_Stuff_Plus'] = ((dch3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dch3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dch3['Final_Adjusted_Stuff_Plus'].mean()

current_std_ct = dct3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_ch
mean = dct3['Stuff_plus'].mean()
dct3['Final_Adjusted_Stuff_Plus'] = ((dct3['Stuff_plus'] - mean) * scaling_factor) + mean
new_std = dct3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dct3['Final_Adjusted_Stuff_Plus'].mean()


current_std_sp = dsp3['Stuff_plus'].std()
desired_std = 10
scaling_factor = desired_std / current_std_sp
mean = dsp3['Stuff_plus'].mean()
dsp3['Final_Adjusted_Stuff_Plus'] = ((dsp3['Stuff_plus'] - mean) * scaling_factor) + mean




/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/4007316141.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/4007316141.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zk/xz_77pl12bx126zpb7kszl180000gn/T/ipykernel_56196/4007316141.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [49]:
#combine all the dataframes
df_2022 = pd.concat([dfb2, dsi2, dsl2, dst2, dcb2, dch2, dct2], ignore_index=True)
df_2023_total = pd.concat([dfb3, dsi3, dsl3, dst3, dcb3, dch3, dct3], ignore_index=True)
#groupby pitcher and TaggedPitchType
df_2023_total_pitch_RV = df_2023_total.groupby(['Pitcher', 'TaggedPitchType']).agg({'RelSpeed': 'mean','InducedVertBreak':'mean','HorzBreak':'mean','Final_Adjusted_Stuff_Plus': 'mean', 'RunValue': 'sum', 'launch_speed':'mean' }).reset_index()
df_2023_total_pitcher_RV = df_2023_total.groupby(['Pitcher']).agg({'Final_Adjusted_Stuff_Plus': 'mean', 'RunValue': 'sum', 'launch_speed':'mean' }).reset_index()
df_2023_total_pitcher_RV['Count'] = df_2023_total.groupby(['Pitcher']).size().reset_index(name='Count')['Count']

In [60]:
from sklearn.linear_model import LinearRegression
df_2023_total_pitcher_RV_count_500 = df_2023_total_pitcher_RV[df_2023_total_pitcher_RV['Count'] > 250]
X = df_2023_total_pitcher_RV_count_500[['Final_Adjusted_Stuff_Plus']]
y = df_2023_total_pitcher_RV_count_500['RunValue']
lm = LinearRegression()
lm.fit(X, y)

y_pred = lm.predict(X)
#r^2
r2 = r2_score(y, y_pred)
print('Unweighted Stuff+ R^2:', r2)

Unweighted Stuff+ R^2: 0.12319408122603681
